In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.nn import relu
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()

In [ ]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target
scaler = StandardScaler()

In [ ]:
X = np.array(X, dtype='float32')
y = np.array(y, dtype='float32')

In [ ]:
y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_train

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

# tf.Variable(initializer([idx, col])) : idx줄, col칸의 variable를 생성해서 반환
w = tf.Variable(initializer([30, 30]))
w

In [ ]:
b = tf.Variable(initializer([30]))
b

In [ ]:
# hidden_layer의 w, b 값
w1 = tf.Variable(initializer([30, 10]))
b1 = tf.Variable(initializer([10]))

# output_layer의 w, b 값
w2 = tf.Variable(initializer([10, 1]))
b2 = tf.Variable(initializer([1]))

In [ ]:
tf.matmul(X_train, w) + b

In [ ]:
hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)

In [ ]:
hypothesis

In [ ]:
# hidden_layer
hypothesis1 = tf.sigmoid(tf.matmul(hypothesis, w1) + b1)
# output_layer
hypothesis2 = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)
hypothesis1, hypothesis2

In [ ]:
optimizer = tf.train.AdamOptimizer(0.01)

In [ ]:
for step in range(1001) :
	with tf.GradientTape() as tape :
		# input_layer
		hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
		# hidden_layer
		hypothesis1 = tf.sigmoid(tf.matmul(hypothesis, w1) + b1)
		# output_layer
		hypothesis2 = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)

		# 선형회귀식의 오차를 계산하는 식
		cost = -tf.reduce_mean(y_train * tf.log(hypothesis2) + (1 - y_train) * tf.log(1 - hypothesis2))

		# 기울기 계산식
		grads = tape.gradient(cost, [w, w1, w2, b, b1, b2])

	# 갱신된 기울기 값을 각각 w, w1, w2, b, b1, b2에 대입
	optimizer.apply_gradients(grads_and_vars=zip(grads, [w, w1, w2, b, b1, b2]))

	if step % 100 == 0 :
		print('=' * 100)
		print(f'step = {step}, cost = {cost.numpy()}')
		print('=' * 100)

In [ ]:
w, b

In [ ]:
w1, b1

In [ ]:
w2, b2

In [ ]:
# 검증
X_test = scaler.transform(X_test)

In [ ]:
hypothesis = tf.sigmoid(tf.matmul(X_test, w) + b)
hypothesis1 = tf.sigmoid(tf.matmul(hypothesis, w1) + b1)
predict = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)
predict

In [ ]:
predict01 = tf.cast(predict > 0.5, 1, 0)
print('=' * 100)
print('predict01')
print(predict01)

In [ ]:
ac01 = tf.equal(predict01, y_test)
ac01

In [ ]:
ac02 = tf.cast(ac01, dtype='float32')
ac02

In [ ]:
ac03 = tf.reduce_mean(ac02)
ac03

In [ ]:
# 위의 ac01 ~ ac03까지의 과정을 one_line으로 표현
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict01, y_test), dtype='float32'))
print('=' * 40)
print('accuracy')
print(accuracy.numpy())